In [8]:
import joblib
hotpot = joblib.load('../agent/hotpot-qa-distractor-sample.joblib').reset_index(drop=True)
hotpot.head()

,id,question,answer,type,level,supporting_facts,context
0,5a7613c15542994ccc9186bf,VIVA Media AG changed it's name in 2004. What ...,Gesellschaft mit beschränkter Haftung,bridge,hard,"{'title': ['VIVA Media', 'Gesellschaft mit bes...","{'title': ['Constantin Medien', 'VIVA Poland',..."
1,5adf2fa35542993344016c11,Which of Jonny Craig and Pete Doherty has been...,"Jonny"" Craig",comparison,hard,"{'title': ['Jonny Craig', 'Jonny Craig', 'Pete...","{'title': ['Pete Doherty', 'Relativity (Emaros..."
2,5adfdef9554299025d62a36b,Where was the first governor after the The Mis...,"Bath, Maine",bridge,hard,"{'title': ['Maine gubernatorial election, 1820...","{'title': ['Compromise of 1790', 'Anti-Nebrask..."
3,5a7180205542994082a3e856,"The creator of ""Wallace and Gromit"" also creat...",Creature Comforts,bridge,hard,"{'title': ['Creature Comforts', 'Creature Comf...","{'title': ['Creature Comforts', 'Tata Steel Zo..."
4,5a78bc6b554299148911f979,Woman's Era and Naj are what kind of magazines?,fortnightly women interest magazine,comparison,hard,"{'title': ['Woman's Era', 'Naj'], 'sent_id': [...","{'title': ['Lifestyle trends and media', 'Chin..."


In [1]:
from discussion_agents.cog.agent.reflexion import ReflexionReActAgent
from langchain_community.chat_models import ChatOpenAI

import dotenv
import os

dotenv.load_dotenv("../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_key=openai_api_key)

question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"
key = "Gesellschaft mit beschränkter Haftung"

agent = ReflexionReActAgent(
    self_reflect_llm=llm,
    action_llm=llm,
    max_steps=7,
    max_trials=3,
)

In [2]:
from typing import List 

def gather_experience(
    reflexion_react_agent: ReflexionReActAgent,
    questions: List[str],
    keys: List[str],
    strategy: str = "reflexion",
):
    out = {
        "idxs": [],
        "questions": [],
        "keys": [],
        "trajectories": [],
        "reflections": []
    }
    for idx, (question, key) in enumerate(zip(questions, keys)):
        out["idxs"].append(idx)
        out["questions"].append(question)
        out["keys"].append(key)
        trajectory = reflexion_react_agent.generate(
            question=question, key=key, strategy=strategy
        )
        out["trajectories"].append(trajectory)
        out["reflections"].append(agent.reflector.reflections)
    return out

In [3]:
out = gather_experience(agent, questions=[question], keys=[key])
out

{'idxs': [0],
 'questions': ["VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"],
 'keys': ['Gesellschaft mit beschränkter Haftung'],
 'trajectories': [[(False,
    '',
    "\nThought: I need to search for VIVA Media AG to find out what their new acronym stands for.\nAction: Search[VIVA Media AG]\nObservation 1: Could not find [VIVA Media AG]. Similar: ['MTV Music (Polish TV channel)', 'VIVA Plus', 'Paramount International Networks', 'VIVA (German TV channel)', 'Viacom (1952–2005)', 'Vauxhall Viva', 'GfK Entertainment charts', 'Lindt', 'Spellbound Entertainment', 'List of multinational corporations']\nThought: I couldn't find VIVA Media AG. Maybe I should try searching for VIVA Media AG name change in 2004 to get the new acronym.\nAction: Search[VIVA Media AG name change 2004]\nObservation 2: Could not find [VIVA Media AG name change 2004]. Similar: ['Vauxhall Viva', 'GfK Entertainment charts', 'Opel Astra', 'Puma (brand)', 'About You Now', 'Bosch (compan

2